All beginning are hard

In [ ]:
import numpy as np

It works in the end

In [ ]:
from rocketpy import Environment, SolidMotor, Rocket, Flight

In [ ]:
env = Environment(latitude=32.990254, longitude=-106.974998, elevation=1400)

In [ ]:
import datetime

tomorrow = datetime.date.today() + datetime.timedelta(days=1)

env.set_date(
    (tomorrow.year, tomorrow.month, tomorrow.day, 12)
)  # Hour given in UTC time

In [ ]:
env.set_atmospheric_model(type="Forecast", file="GFS")

In [ ]:
env.info()

In [ ]:
from math import exp
from rocketpy import Fluid, LiquidMotor, CylindricalTank, MassFlowRateBasedTank

In [ ]:
# Define fluids
oxidizer_liq = Fluid(name="N2O_l", density=1220)
oxidizer_gas = Fluid(name="N2O_g", density=1.9277)
fuel_liq = Fluid(name="ethanol_l", density=789)
fuel_gas = Fluid(name="ethanol_g", density=1.59)

# Define tanks geometry
tanks_shape = CylindricalTank(radius = 0.1, height = 1.2, spherical_caps = True)

# Define tanks
oxidizer_tank = MassFlowRateBasedTank(
    name="oxidizer tank",
    geometry=tanks_shape,
    flux_time=5,
    initial_liquid_mass=32,
    initial_gas_mass=0.01,
    liquid_mass_flow_rate_in=0,
    liquid_mass_flow_rate_out=lambda t: 32 / 3 * exp(-0.25 * t),
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out=0,
    liquid=oxidizer_liq,
    gas=oxidizer_gas,
)

fuel_tank = MassFlowRateBasedTank(
    name="fuel tank",
    geometry=tanks_shape,
    flux_time=5,
    initial_liquid_mass=21,
    initial_gas_mass=0.01,
    liquid_mass_flow_rate_in=0,
    liquid_mass_flow_rate_out=lambda t: 21 / 3 * exp(-0.25 * t),
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out=lambda t: 0.01 / 3 * exp(-0.25 * t),
    liquid=fuel_liq,
    gas=fuel_gas,
)


In [ ]:
example_liquid = LiquidMotor(
    thrust_source=lambda t: 4000,
    dry_mass=2,
    dry_inertia=(0.125, 0.125, 0.002),
    nozzle_radius=0.075,
    center_of_dry_mass_position=1.75,
    nozzle_position=0,
    burn_time=5,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)
example_liquid.add_tank(tank=oxidizer_tank, position=1.0)
example_liquid.add_tank(tank=fuel_tank, position=2.5)

In [ ]:
example_liquid.info()

In [ ]:
example_liquid.all_info()

In [ ]:
calisto = Rocket(
    radius=150 / 2000,
    mass=1.426,
    inertia=(6.321, 6.321, 0.034),
    power_off_drag="C:/Users/krikb/Desktop/simulations/powerOffDragCurve.csv",
    power_on_drag="C:/Users/krikb/Desktop/simulations/powerOnDragCurve.csv",
    center_of_mass_without_motor=0,
    coordinate_system_orientation="tail_to_nose",
)

In [ ]:
calisto.add_motor(example_liquid, position=-1.255)

In [ ]:
rail_buttons = calisto.set_rail_buttons(
    upper_button_position=0.0818,
    lower_button_position=-0.6182,
    angular_position=45,
)

In [ ]:
nose_cone = calisto.add_nose(
    length=0.55829, kind="von karman", position=1.278
)

fin_set = calisto.add_trapezoidal_fins(
    n=4,
    root_chord=0.120,
    tip_chord=0.060,
    span=0.110,
    position=-1.04956,
    cant_angle=0.5,
)

tail = calisto.add_tail(
    top_radius=0.0635, bottom_radius=0.0435, length=0.060, position=-1.194656
)

In [ ]:
main = calisto.add_parachute(
    name="main",
    cd_s=10.0,
    trigger=800,      # ejection altitude in meters
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
    radius=1.5,
    height=1.5,
    porosity=0.0432,
)

drogue = calisto.add_parachute(
    name="drogue",
    cd_s=1.0,
    trigger="apogee",  # ejection at apogee
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
    radius=1.5,
    height=1.5,
    porosity=0.0432,
)

In [ ]:
calisto.plots.static_margin()

In [ ]:
calisto.draw()


In [ ]:
test_flight = Flight(
    rocket=calisto, environment=env, rail_length=5.2, inclination=85, heading=0
    )

In [ ]:
test_flight.prints.initial_conditions()

In [ ]:
test_flight.prints.out_of_rail_conditions()

In [ ]:
test_flight.prints.burn_out_conditions()

In [ ]:
test_flight.prints.apogee_conditions()

In [ ]:
test_flight.prints.events_registered()

In [ ]:
test_flight.prints.impact_conditions()

In [ ]:
test_flight.prints.maximum_values()

In [ ]:
test_flight.plots.trajectory_3d()